In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from tensorflow.python.keras.callbacks import TensorBoard, ReduceLROnPlateau, EarlyStopping
import sklearn
from numpy import mean
from numpy import std
import random as python_random
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import seaborn as sns
from keras.constraints import maxnorm
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from keras.layers import Dense, Dropout
from time import time

In [ ]:
df = pd.read_csv('/content/train.csv',low_memory=False,delimiter=',', header=None
                  ,skiprows=1, names=['Year','Month','Day','Hour','Minute','Clearsky DHI','Clearsky DNI','Clearsky GHI','Cloud Type','Dew Point','Temperature','Pressure','Relative Humidity',
       'Solar Zenith Angle','Precipitable Water','Wind Direction','Wind Speed','Fill Flag'])

In [ ]:
sns.relplot(x='Clearsky DHI',y='Temperature',hue='Wind Speed',data=df)
sns.relplot(x='Clearsky DNI',y='Temperature',hue='Wind Speed',data=df)
sns.relplot(x='Clearsky GHI',y='Temperature',hue='Wind Speed',data=df)

In [ ]:
sns.catplot(x='Clearsky GHI',kind='box',data=df)

In [ ]:
sns.heatmap(df.corr(),vmin=-1,vmax=1,cmap="Blues")

In [ ]:
seed_value=12321
np.random.seed(seed_value)
Name='neural_network-{}'.format(int(time()))
tensorboard = TensorBoard(log_dir='logs/{}'.format(Name))
reduce_lr=ReduceLROnPlateau(monitor='val_loss',factor=0.2,patience=0.5,min_lr=0.001)
es=EarlyStopping(monitor='val_loss',patience=25,verbose=2)

X = df[['Year','Month','Day','Hour','Minute','Cloud Type','Dew Point','Temperature','Pressure','Relative Humidity',
       'Solar Zenith Angle','Precipitable Water','Wind Direction','Wind Speed','Fill Flag']]
y = df[['Clearsky DHI','Clearsky DNI','Clearsky GHI']]

In [ ]:
# Visualizing the data
print(X.info())
print(y.info())

In [ ]:
# Delete all the rows that have null values if the all the values were null in that specific row
X.dropna(axis='index', how='all')
X

In [ ]:
##### split data
x_train, x_test, y_train, y_test = train_test_split(X, y,#splits data into train and test sets
    train_size=0.8,test_size=0.2,random_state=42)
    
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, #splits data in training set into val set
test_size=0.25,train_size=0.75,random_state=42)
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)
print(x_val.shape,y_val.shape)

In [ ]:
# Normalize the data
scaler=MinMaxScaler()
scaler.fit(x_train)
x_train=scaler.transform(x_train)
x_val=scaler.transform(x_val)
x_test=scaler.transform(x_test)

print(x_train)
print(x_val)

In [ ]:
model=Sequential()
model.add(Dense(128,input_dim=15,kernel_initializer='he_uniform',activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(3))
model.compile(loss='mean_squared_error',optimizer='adam',metrics=['accuracy'])


In [ ]:
history=model.fit(x_train,y_train,epochs=500,batch_size=64,validation_data=[x_val,y_val],
                  callbacks=[tensorboard,es],verbose=2)

In [ ]:
#####loss plot
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper right')
plt.show()

#####accuracy plot
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper right')
plt.show()

In [ ]:
results=model.evaluate(x_test,y_test,batch_size=128)
print("test loss, test acc:", results)